In [8]:
import numpy as np
import torch
import torch.nn.functional as F

In [57]:
# Реализация транспонированной свертки
def convolution_transpose(input_data, kernel, stride=1):
    input_channels, input_height, input_width = input_data.shape
    kernel_channels, kernel_height, kernel_width = kernel.shape
    
    output_height = (input_height - 1) * stride + kernel_height
    output_width = (input_width - 1) * stride + kernel_width
    output = np.zeros((input_channels, output_height, output_width))
    
    for c in range(input_channels):
        for i in range(input_height):
            for j in range(input_width):
                output[c, i * stride:i * stride + kernel_height, j * stride:j * stride + kernel_width] += kernel[c] * input_data[c, i, j]
    
    return output

In [48]:
# Реализация транспонированной свертки методом билинейной интерполяции
def transpose_conv2D(input, weight, bias=None, stride=1, padding=0, output_padding=0, groups=1, dilation=1):
    # Получение выходных размеров и пустого массива для результатов
    batch_size, in_channels, img_height, img_width = input.shape
    out_channels, in_channels, kernel_height, kernel_width = weight.shape

    # Вычисление размеров выходных данных после транспонированной свертки
    h_out = (img_height - 1) * stride - 2 * padding + kernel_height + output_padding
    w_out = (img_width - 1) * stride - 2 * padding + kernel_width + output_padding

    # Создание пустого массива для хранения результатов
    result = np.zeros((batch_size, out_channels, h_out, w_out))

    # Проверка и применение заполнения, если оно задано
    if padding > 0:
        input = np.pad(input, padding, mode='constant')

    batch_size, out_channels, H_out, W_out = result.shape

    # Циклы для обработки каждого пикселя выходного изображения
    for batch in range(batch_size):
        for channel_o in range(out_channels):
            for h_o in range(H_out):
                for w_o in range(W_out):
                    for in_c in range(input.shape[1]):
                        for kern_h in range(weight.shape[2]):
                            for kern_w in range(weight.shape[3]):
                                # Вычисление координат входного пикселя
                                ii = h_o + padding - kern_h * dilation
                                jj = w_o + padding - kern_w * dilation

                                # Проверка попадания в границы входных данных
                                if ii >= 0 and jj >= 0 and ii < input.shape[2] * stride and jj < input.shape[3] * stride and (ii % stride == 0) and (jj % stride == 0):
                                    ii //= stride
                                    jj //= stride
                                    # Выполнение транспонированной свертки и накопление результата
                                    result[batch, channel_o, h_o, w_o] += np.multiply(input[batch, in_c, ii, jj], weight[channel_o, in_c, kern_h, kern_w])
    return result

In [51]:
def test_compare_with_torch():
    image = torch.randn(1, 1, 5, 5) 
    weight = torch.randn(1, 1, 3, 3)
        
    myConv2D_T = torch.from_numpy(transpose_conv2D(image.numpy(), weight.numpy()))


    torchConv2D_T = F.conv_transpose2d(image, weight)

    myConv2D_T = myConv2D_T.to(torchConv2D_T.dtype)

    # Сравнение результатов
    if np.allclose(myConv2D_T, torchConv2D_T):
        print("Результаты совпадают: Успех!")
    else:
        print("Результаты НЕ совпадают")

test_compare_with_torch()

Результаты совпадают: Успех!


In [52]:
def test_compare_with_torch():
    image = torch.randn(1, 1, 5, 5) 
    weight = torch.randn(1, 1, 3, 3)
        
    myConv2D_T = torch.from_numpy(transpose_conv2D(image.numpy(), weight.numpy()))


    torchConv2D_T = F.conv_transpose2d(image, weight)

    myConv2D_T = myConv2D_T.to(torchConv2D_T.dtype)

    # Сравнение результатов
    if np.allclose(myConv2D_T, torchConv2D_T):
        print("Результаты совпадают: Успех!")
    else:
        print("Результаты НЕ совпадают")

test_compare_with_torch()

Результаты совпадают: Успех!


In [56]:
def test_compare_with_torch():
    image = torch.randn(1, 1, 5, 5) 
    weight = torch.randn(1, 1, 3, 3)
        
    myConv2D_T = torch.from_numpy(transpose_conv2D(image.numpy(), weight.numpy()))


    torchConv2D_T = F.conv_transpose2d(image, weight)

    myConv2D_T = myConv2D_T.to(torchConv2D_T.dtype)

    # Сравнение результатов
    if np.allclose(myConv2D_T, torchConv2D_T):
        print("Результаты совпадают: Успех!")
    else:
        print("Результаты НЕ совпадают")

test_compare_with_torch()

Результаты совпадают: Успех!
